In [13]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [14]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [15]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [16]:
launch_sites = spacex_df['Launch Site'].unique()
launch_sites

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [17]:
dropdownSites = [{'label': 'All Sites', 'value': 'ALL'}]

for value in launch_sites:
    dropdownSites.append({'label': str(value), 'value': str(value)})

In [18]:
filtered_dfTest = spacex_df[['Launch Site','class']]
filtered_dfTest = filtered_dfTest.where(filtered_dfTest['Launch Site'] == 'CCAFS LC-40')
filtered_dfTest.dropna(axis=0, inplace=True)
filtered_dfTest.groupby('class').count().reset_index()

,class,Launch Site
0,0.0,19
1,1.0,7


In [19]:
spacex_df['Payload Mass (kg)'].min()

0.0

In [20]:
min([1,5])

1

In [21]:
# Create a dash application
app = dash.Dash(__name__)

In [22]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=dropdownSites,
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                            ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()],
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [23]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[['Launch Site','class']]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = filtered_df.where(filtered_df['Launch Site'] == str(entered_site))
        filtered_df.dropna(axis=0, inplace=True)
        filtered_df = filtered_df.groupby('class').count().reset_index()
        fig = px.pie(filtered_df, values='Launch Site', 
        names='class', 
        title='Total Success Launches By Site')
        return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')])
def get_scatter_plot(entered_site, entered_range):
    filtered_df = spacex_df[['Booster Version','Payload Mass (kg)','class']]
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] > min(entered_range)) & (filtered_df['Payload Mass (kg)'] < max(entered_range))]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
        return fig
    else:
        filtered_df = filtered_df.where(filtered_df['Launch Site'] == str(entered_site))
        filtered_df.dropna(axis=0, inplace=True)
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class')
        return fig


In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1632235559.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1632235559.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_0m1632235559.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1632235559.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 22:58:57] "GET /_dash-component-sui

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Vitto\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch Site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
 

127.0.0.1 - - [17/Oct/2021 23:01:28] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Oct/2021 23:01:29] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Vitto\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch Site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
 

...

127.0.0.1 - - [17/Oct/2021 23:01:30] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Oct/2021 23:01:31] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Vitto\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch Site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
 

127.0.0.1 - - [17/Oct/2021 23:01:31] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Oct/2021 23:01:33] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Vitto\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch Site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Vitto\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
 

127.0.0.1 - - [17/Oct/2021 23:01:33] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Oct/2021 23:02:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 23:02:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 23:03:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2021 23:03:14] "POST /_dash-update-component HTTP/1.1" 200 -
